Things to try to increase performance: 
-even more data
-separate models for different genres
-topic model
-ridge regression, lasso regression
-bi grams, n grams
-sentiment analysis
-Look for transformations opps, etc
-model only using genre

In [144]:
import pandas as pd
from textblob import TextBlob
from textblob import Word
from gensim import corpora, models, similarities
import os
import tempfile
import gensim
from sklearn.linear_model import LinearRegression
import math
import numpy as np
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from sklearn.model_selection import train_test_split
import itertools

TEMP_FOLDER = tempfile.gettempdir()

In [2]:
links = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/links2.csv', header = None)
text = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/reviewtext.csv', header = None)

In [3]:
links2 = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/links3.csv', header = None)
text2 = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/reviewtext2.csv', header = None)

In [4]:
#prep 2nd set of data, which has lots of links that didn't work

links2[5] = text2
links2.drop_duplicates([5], inplace = True)
links2 = links2[links2[5] != '[]']
links2 = links2[links2[5] != '[]']
links2.dropna(subset = [5],inplace = True)

In [5]:
links[5] = text

In [6]:
links.drop_duplicates([5], inplace = True)


In [7]:
links = pd.concat([links,links2], axis = 0)
links = links.reset_index()
links = links.drop('index', axis = 1)

In [13]:
links.shape

(2903, 6)

In [14]:
text = links[5]
text.shape

(2903,)

In [15]:
#lowercase text
text = text.apply(lambda x: " ".join(x.lower() for x in x.split()))
text.head()

0    ["with all of the development muscle behind ea...
1    ['when i first reviewed civilization vi back i...
2    [' ', 'there aren’t many games that make me fe...
3    ['overkill’s ', ' is an earnest attempt to del...
4    ["it's often with rose-colored glasses that i ...
Name: 5, dtype: object

In [16]:
#remove brackets
text = text.apply(lambda x: x[1:-1])
text.head()

0    "with all of the development muscle behind ea ...
1    'when i first reviewed civilization vi back in...
2    ' ', 'there aren’t many games that make me fee...
3    'overkill’s ', ' is an earnest attempt to deli...
4    "it's often with rose-colored glasses that i l...
Name: 5, dtype: object

In [17]:
#remove new line
text = text.replace(r'\\n','', regex=True)
text.head()

0    "with all of the development muscle behind ea ...
1    'when i first reviewed civilization vi back in...
2    ' ', 'there aren’t many games that make me fee...
3    'overkill’s ', ' is an earnest attempt to deli...
4    "it's often with rose-colored glasses that i l...
Name: 5, dtype: object

In [18]:
#remove punc
text = text.str.replace('[^\w\s]','')
text.head()

0    with all of the development muscle behind ea d...
1    when i first reviewed civilization vi back in ...
2      there arent many games that make me feel as ...
3    overkills   is an earnest attempt to deliver a...
4    its often with rosecolored glasses that i look...
Name: 5, dtype: object

In [19]:
#remove stop words
from nltk.corpus import stopwords
stop = stopwords.words('english')
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
text.head()

0    development muscle behind ea dices massive sho...
1    first reviewed civilization vi back 2016 award...
2    arent many games make feel cool warframe im bu...
3    overkills earnest attempt deliver cooperative ...
4    often rosecolored glasses look back pokemons k...
Name: 5, dtype: object

In [20]:
freq = pd.Series(' '.join(text[0]).split()).value_counts()[-5:]
#remove uncommon words
freq = list(freq.index)
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
text.head()

0    development muscle behind ea dices massive sho...
1    first reviewed civilization vi back 2016 award...
2    arent many games make feel cool warframe im bu...
3    overkills earnest attempt deliver cooperative ...
4    often rosecolored glasses look back pokemons k...
Name: 5, dtype: object

In [21]:
###remove common words###
freq = pd.Series(' '.join(text[0]).split()).value_counts()[:10]
freq = list(freq.index)
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
#Visualize
text.head()

0    development muscle behind ea dices massive sho...
1    first reviewed civilization vi back 2016 award...
2    arent many games make feel cool warframe im bu...
3    overkills earnest attempt deliver cooperative ...
4    often rosecolored glasses look back pokemons k...
Name: 5, dtype: object

In [22]:
###Spelling correction###

text[:5].apply(lambda x: str(TextBlob(x).correct()))

###Tokenization (dividing the text into a sequence of words or sentences)###
TextBlob(text[0][1]).words

#Stemming (removal of suffices, like “ing”, “ly”, “s”, etc. by a simple rule-based approach)
#from nltk.stem import PorterStemmer
#st = PorterStemmer()
#train['text'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

#Lemmatization (more effective option than stemming because 
# converts the word into its root word, rather than just stripping the suffices)
#*Preferred to stemming*#


text = text.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
text.head()

0    development muscle behind ea dice massive shoo...
1    first reviewed civilization vi back 2016 award...
2    arent many game make feel cool warframe im bul...
3    overkill earnest attempt deliver cooperative a...
4    often rosecolored glass look back pokemons kan...
Name: 5, dtype: object

In [23]:
text = text.to_frame()
text['score'] = links[1]

In [24]:
text.head()

,5,score
0,development muscle behind ea dice massive shoo...,7.5
1,first reviewed civilization vi back 2016 award...,9.4
2,arent many game make feel cool warframe im bul...,8.6
3,overkill earnest attempt deliver cooperative a...,5.3
4,often rosecolored glass look back pokemons kan...,8.3


In [258]:
#train val split

trainval = text.sample(800)
train = text.loc[~text.index.isin(trainval.index), :]
y = train['score']
valy = trainval['score']
trainval.drop(['score'], axis = 1, inplace = True)
train.drop(['score'], axis = 1, inplace = True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [259]:
#for train values
textlist = train.values.tolist()

In [260]:
textlist = [str(word) for word in textlist]

In [261]:
texts = [[word for word in document.lower().split() ]
         for document in textlist]

In [262]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=5, no_above=0.6, keep_n=10000)

In [263]:
dictionary.save(os.path.join(TEMP_FOLDER, 'igntfidf.dict'))

In [264]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'igntfidf.mm'), corpus)  # store to disk, for later use

In [265]:
tfidf = models.TfidfModel(corpus)

In [266]:
corpus_tfidf = tfidf[corpus]

In [267]:
all_topics_csr = gensim.matutils.corpus2csc(corpus_tfidf)

In [268]:
all_topics_csr.shape

(10000, 2103)

In [269]:
all_topics_numpy = all_topics_csr.T.toarray()

In [270]:
all_topics_numpy

array([[0.0657612 , 0.03284519, 0.15750011, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.01415388, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.01877435, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [271]:
#validation array
vallist = trainval.values.tolist()
vallist = [str(word) for word in vallist]
valtexts = [[word for word in document.lower().split() ]
         for document in vallist]
valcorpus = [dictionary.doc2bow(valtext) for valtext in valtexts]
testcorpus_tfidf = tfidf[valcorpus]
all_topics_csr2 = gensim.matutils.corpus2csc(testcorpus_tfidf)
all_topics_numpytest = all_topics_csr2.T.toarray()

In [272]:
all_topics_numpytest

array([[0.        , 0.        , 0.        , ..., 0.1401632 , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.14835579, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [273]:
print(all_topics_numpytest.shape)
print(all_topics_numpy.shape)

(800, 10000)
(2103, 10000)


In [274]:
y.tail()

2896    6.0
2897    8.0
2899    5.0
2900    7.0
2901    7.5
Name: score, dtype: float64

In [278]:
train = pd.DataFrame(all_topics_numpy)
y = y.reset_index()
y.drop(['index'], axis = 1, inplace = True)
train['target'] = y



test = pd.DataFrame(all_topics_numpytest)
valy = valy.reset_index()
valy.drop(['index'], axis = 1, inplace = True)
test['target'] = valy
type(train)

pandas.core.frame.DataFrame

In [279]:

col_train = list(train.columns)
col_train = [(str(x)+'col')for x in col_train]
col_train_bis = list(train.columns)
col_train_bis = [(str(x)+'col')for x in col_train_bis]
train.columns = col_train
test.columns = col_train

col_train_bis.remove('targetcol')
# List of features
COLUMNS = col_train
FEATURES = col_train_bis
LABEL = "targetcol"

# Columns for tensorflow
feature_cols = [tf.contrib.layers.real_valued_column(k) for k in FEATURES]

In [246]:
tf.logging.set_verbosity(tf.logging.ERROR)
regressor = tf.contrib.learn.DNNRegressor(feature_columns=feature_cols, 
                                          activation_fn = tf.nn.relu, hidden_units=[200, 100, 50, 25, 12])#,
                                         #optimizer = tf.train.GradientDescentOptimizer( learning_rate= 0.1 ))

In [247]:
train2.reset_index(drop = True, inplace =True)

In [248]:
def input_fn(data_set, pred = False):
    
    if pred == False:
        
        feature_cols = {k: tf.constant(data_set[k].values) for k in FEATURES}
        labels = tf.constant(data_set[LABEL].values)
        
        return feature_cols, labels

    if pred == True:
        feature_cols = {k: tf.constant(data_set[k].values) for k in FEATURES}
        
        return feature_cols

In [249]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [250]:
#let's try reducing the size of the tfidf first, then we will work on retraining

In [251]:
regressor.fit(input_fn=lambda: input_fn(train2), steps=2000)

DNNRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x1c73287b70>, 'hidden_units': [200, 100, 50, 25, 12], 'feature_columns': (_RealValuedColumn(column_name='0col', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='1col', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='2col', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='3col', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='4col', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='5col', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='6col', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='7col', dimension=1, default_value=N

In [252]:
y = regressor.predict(input_fn=lambda: input_fn(test))

In [253]:
predictions = list(itertools.islice(y, testing_set.shape[0]))

In [254]:
predictionsdf = pd.DataFrame(predictions)
predictionsdf.describe()

,0
count,694.000000
mean,7.141957
std,1.230353
min,3.113450
25%,6.406502
50%,7.304657
75%,7.983594
max,10.846616


In [255]:
predictionsdf = pd.DataFrame(predictions)
valy = valy.reset_index()
valy.drop(['index'], axis = 1, inplace = True)
predictionsdf['true'] = valy
predictionsdf['diff'] = predictionsdf['true'] - predictionsdf[0]
predictionsdf['diff'] = predictionsdf['true'] - predictionsdf[0]
predictionsdf['diff'] = predictionsdf['diff']**2
diff = predictionsdf['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff

1.0153354758473956

## XGBoost

In [344]:
#bayesian opt
yarray = y.values
dtrain = xgb.DMatrix(all_topics_numpy, label=yarray)
def xgb_evaluate(max_depth, gamma, colsample_bytree):
    params = {'eval_metric': 'rmse',
              'max_depth': int(max_depth),
              'subsample': 0.8,
              'eta': 0.1,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree}
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=3)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7),
                                            'gamma': (0, 1),
                                            'colsample_bytree': (0.3, 0.9)})

xgb_bo.maximize(init_points=3, n_iter=5, acq='ei')



|   iter    |  target   | colsam... |   gamma   | max_depth |
-------------------------------------------------------------
[19:39:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=4
[19:39:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=4
[19:39:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=4
[19:39:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=4
[19:39:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=4
[19:39:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 2 pruned nodes, max_depth=4
[19:39:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 2 pruned nodes, max_depth=4
[19:39:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned

[19:39:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 4 pruned nodes, max_depth=4
[19:39:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 6 pruned nodes, max_depth=4
[19:39:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 2 pruned nodes, max_depth=4
[19:39:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 6 pruned nodes, max_depth=4
[19:39:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[19:39:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 2 pruned nodes, max_depth=4
[19:39:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=4
[19:39:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[19:39:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_

[19:40:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 4 pruned nodes, max_depth=4
[19:40:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 2 pruned nodes, max_depth=4
[19:40:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 4 pruned nodes, max_depth=4
[19:40:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 2 pruned nodes, max_depth=4
[19:40:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 4 pruned nodes, max_depth=4
[19:40:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 2 pruned nodes, max_depth=4
[19:40:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 2 pruned nodes, max_depth=4
[19:40:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[19:40:11] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 6 pruned nodes, max_

[19:40:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 4 pruned nodes, max_depth=4
[19:40:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 2 pruned nodes, max_depth=4
[19:40:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 2 pruned nodes, max_depth=4
[19:40:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 2 pruned nodes, max_depth=4
[19:40:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 6 pruned nodes, max_depth=4
[19:40:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 6 pruned nodes, max_depth=4
[19:40:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[19:40:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 2 pruned nodes, max_depth=4
[19:40:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 4 pruned nodes, max_

[19:40:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 14 pruned nodes, max_depth=4
[19:40:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 2 pruned nodes, max_depth=4
[19:40:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 8 pruned nodes, max_depth=4
[19:40:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 4 pruned nodes, max_depth=4
[19:40:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 2 pruned nodes, max_depth=4
|  1        | -1.108    |  0.5575   |  0.5564   |  4.346    |
[19:40:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:40:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:40:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:40:50] src/tree/updater_prune.cc:74: 

...

[19:47:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 12 pruned nodes, max_depth=4
[19:47:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=4
[19:47:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 14 pruned nodes, max_depth=4
[19:47:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 6 pruned nodes, max_depth=4
[19:47:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 16 pruned nodes, max_depth=4
[19:47:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 8 pruned nodes, max_depth=4
[19:47:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 12 pruned nodes, max_depth=4
[19:47:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 4 pruned nodes, max_depth=4
[19:47:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 2 pruned nodes, 

[19:48:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_depth=3
[19:48:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:48:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:48:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:48:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:48:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:48:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:48:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:48:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_

[19:48:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:48:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:48:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_depth=3
[19:48:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_depth=3
[19:48:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:48:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_depth=3
[19:48:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:48:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:48:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_

[19:49:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:49:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_depth=3
[19:49:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:49:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:49:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_depth=3
[19:49:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_depth=3
[19:49:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 4 pruned nodes, max_depth=3
[19:49:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_depth=3
[19:49:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_

[19:49:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 2 pruned nodes, max_depth=3
[19:49:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 4 pruned nodes, max_depth=3
[19:49:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 2 pruned nodes, max_depth=3
[19:49:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:49:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 4 pruned nodes, max_depth=3
[19:49:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:49:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 2 pruned nodes, max_depth=3
[19:49:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 4 pruned nodes, max_depth=3
[19:49:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_d

In [357]:
param = xgb_bo.max
param

{'target': -1.1006746666666667,
 'params': {'colsample_bytree': 0.8976729774092942,
  'gamma': 0.9971962663965198,
  'max_depth': 4.321612175930944}}

In [340]:
yarray = y.values
xgbreg = xgb.XGBRegressor()
xgbreg.fit(all_topics_numpy,yarray)

predictions2 = xgbreg.predict(all_topics_numpytest)
predictionsdf2 = pd.DataFrame(predictions2)
valy = valy.reset_index()
valy.drop(['index'], axis = 1, inplace = True)
predictionsdf2['true'] = valy
predictionsdf2['diff'] = predictionsdf2['true'] - predictionsdf2[0]
predictionsdf2['diff'] = predictionsdf2['true'] - predictionsdf2[0]
predictionsdf2['diff'] = predictionsdf2['diff']**2
diff = predictionsdf2['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff

1.0728785941278771

## Lasso Regression

In [338]:
#find best alpha for lasso regression

alphas = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]
alphas2 = [1e-5,1.5e-5,1e-4, 1.5e-4,1e-3]
def runlasso(alpha):
    global valy
    yarray = y.values
    lasso = Lasso(alpha = alpha, max_iter=1e5)
    lasso.fit(all_topics_numpy,yarray)

    predictions3 = lasso.predict(all_topics_numpytest)
    predictionsdf3 = pd.DataFrame(predictions3)
    valy = valy.reset_index()
    valy.drop(['index'], axis = 1, inplace = True)
    predictionsdf3['true'] = valy
    predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
    predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
    predictionsdf3['diff'] = predictionsdf3['diff']**2
    diff = predictionsdf3['diff'].values
    diff = np.mean(diff)
    diff = math.sqrt(diff)
    return diff
    
test = [runlasso(x) for x in alphas2]
print(test)
    
    
    

[1.1252147868577373, 1.1126873118616674, 1.0177467770085127, 0.9907478349117224, 1.1576718275231785]


In [339]:
yarray = y.values
lasso = Lasso(alpha = 1.5e-4, max_iter=1e5)
lasso.fit(all_topics_numpy,yarray)

predictions3 = lasso.predict(all_topics_numpytest)
predictionsdf3 = pd.DataFrame(predictions3)
valy = valy.reset_index()
valy.drop(['index'], axis = 1, inplace = True)
predictionsdf3['true'] = valy
predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
predictionsdf3['diff'] = predictionsdf3['diff']**2
diff = predictionsdf3['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff


0.9907478349117224

## OLS

In [327]:
yarray = y.values
lm = LinearRegression()
lm.fit(all_topics_numpy,yarray)
predictions = lm.predict(all_topics_numpytest)
predictionsdf = pd.DataFrame(predictions)

In [328]:
predictionsdf.describe()

,0
count,800.000000
mean,7.257459
std,1.230349
min,1.965389
25%,6.399265
50%,7.289813
75%,8.135253
max,10.575431


In [329]:
valy = valy.reset_index()
valy

,index,score
0,0,7.7
1,1,8.0
2,2,9.0
3,3,7.8
4,4,9.0
5,5,6.4
6,6,7.9
7,7,9.2
8,8,7.0
9,9,7.0


In [330]:
valy.drop(['index'], axis = 1, inplace = True)
valy

,score
0,7.7
1,8.0
2,9.0
3,7.8
4,9.0
5,6.4
6,7.9
7,9.2
8,7.0
9,7.0


In [331]:
predictionsdf['true'] = valy
predictionsdf

,0,true
0,7.608641,7.7
1,8.287071,8.0
2,8.508838,9.0
3,8.017991,7.8
4,7.571709,9.0
5,6.604391,6.4
6,8.054136,7.9
7,8.822618,9.2
8,7.870244,7.0
9,6.725158,7.0


In [332]:
predictionsdf['diff'] = predictionsdf['true'] - predictionsdf[0]
predictionsdf

,0,true,diff
0,7.608641,7.7,0.091359
1,8.287071,8.0,-0.287071
2,8.508838,9.0,0.491162
3,8.017991,7.8,-0.217991
4,7.571709,9.0,1.428291
5,6.604391,6.4,-0.204391
6,8.054136,7.9,-0.154136
7,8.822618,9.2,0.377382
8,7.870244,7.0,-0.870244
9,6.725158,7.0,0.274842


In [333]:
#let's also plot histogram of scores




In [334]:
#residual plot here




In [335]:
#RMSE
predictionsdf['diff'] = predictionsdf['true'] - predictionsdf[0]
predictionsdf['diff'] = predictionsdf['diff']**2
diff = predictionsdf['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff

#diff = predictionsdf['diff'].values.tolist()
#diff = math.sqrt(np.mean(diff**2))
#diff

1.0528095780414812

In [256]:
yarray = y.values
arravg = np.mean(yarray)
predictionsdf['avg'] = arravg
predictionsdf['avgdiff'] = predictionsdf['avg'] - predictionsdf['true']
predictionsdf['avgdiff'] = predictionsdf['avgdiff']**2
avgdiff = predictionsdf['avgdiff'].values
avgdiff = np.mean(avgdiff)
avgdiff = math.sqrt(avgdiff)
avgdiff

AttributeError: 'generator' object has no attribute 'values'